In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
BASE_DIR = '/kaggle/input/tabular-playground-series-mar-2021/'
train_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'))
test_df = pd.read_csv(os.path.join(BASE_DIR,'test.csv'))

In [ ]:
train_df.head()

In [ ]:
cols = train_df.columns
cols

In [ ]:
cat_cols = [col for col in cols if 'cat' in col]
cont_cols = [col for col in cols if 'cont' in col]
print('cat_cols : ', cat_cols)
print('cont_cols : ', cont_cols)

In [ ]:
train_df[cat_cols].isnull().sum()

In [ ]:
train_df[cont_cols].isnull().sum()

In [ ]:
test_df[cat_cols].isnull().sum()

In [ ]:
test_df[cont_cols].isnull().sum()

In [ ]:
train_df[cat_cols].head()

In [ ]:
trainCopy = train_df.copy()
testCopy = test_df.copy()

# the data

In [ ]:
trainCopy.drop('id', axis=1, inplace=True)
testCopy.drop('id', axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
lb = LabelEncoder()
for data in [trainCopy, testCopy]:
    for col in cat_cols:
        data[col] = pd.DataFrame(lb.fit_transform(data[col]))

# feature and target

In [ ]:
feature = trainCopy.drop('target', axis=1).columns
target = trainCopy['target']

In [ ]:
trainCopy.target.dtype

In [ ]:
trainCopy.target.unique()

In [ ]:
trainCopy[cont_cols].head()

In [ ]:
trainCopy[cat_cols].head()

# Fully connected neural network

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras import models, layers
import tensorflow as tf

In [ ]:
devicename = tf.test.gpu_device_name()
devicename

In [ ]:
tf.device(devicename)

In [ ]:
# X_train_cat = X_train[cat_cols]
# X_train_cont = X_train[cont_cols]
# X_valid_cat = X_valid[cat_cols]
# X_valid_cont = X_valid[cont_cols]
# X_train_cat = X_train_cat / np.max(X_train_cat)
# X_train_cont = X_train_cont / np.max(X_train_cont)
# X_valid_cat = X_valid_cat / np.max(X_valid_cat)
# X_valid_cont = X_valid_cont / np.max(X_valid_cont)

# # x
# inputx = layers.Input(shape=(X_train_cat.shape[1],))
# x = layers.Dense(2*X_train_cat.shape[1])(inputx)
# x = layers.Activation('tanh')(x)
# # x = layers.Dense(100)(x)
# # x = layers.Activation('relu')(x)
# x = models.Model(inputs=inputx, outputs=x)
# # y
# inputy = layers.Input(shape=(X_train_cont.shape[1],))
# y = layers.Dense(2*X_train_cont.shape[1])(inputy)
# y = layers.Activation('tanh')(y)
# # y = layers.Dense(100)(y)
# # y = layers.Activation('relu')(y)
# y = models.Model(inputs=inputy, outputs=y)
# # merge
# merge = layers.concatenate([x.output, y.output])
# z = layers.Dense(38+22)(merge)
# # z = layers.Activation('tanh')(z)
# # z = layers.Dense(800)(z)
# # z = layers.Activation('relu')(z)
# # z = layers.Dropout(0.1)(z)
# z = layers.Dense(1, activation='sigmoid')(z)
# z = models.Model(inputs=[x.input, y.input], outputs=z)

# z.compile(loss='binary_crossentropy',
#          optimizer='adam',
#          metrics=['accuracy'])

# z.fit([X_train_cat, X_train_cont], y_train,
#      validation_data=([X_valid_cat, X_valid_cont], y_valid),
#      batch_size=100,
#      epochs=50)

In [ ]:
# import matplotlib.pyplot as plt
# acc = z.history.history['accuracy']
# val_acc = z.history.history['val_accuracy']
# epochs = range(1, len(acc)+1)
# plt.plot(epochs, acc, 'r', label='train')
# plt.plot(epochs, val_acc, 'b', label='validation')
# plt.legend()

In [ ]:
trainCopy[cat_cols] = trainCopy[cat_cols] / np.max(trainCopy[cat_cols])
testCopy[cat_cols] = testCopy[cat_cols] / np.max(testCopy[cat_cols])
testCopy[cat_cols]

In [ ]:
import optuna
X_train, X_valid, y_train, y_valid = train_test_split(trainCopy[feature], target, test_size=0.25, random_state=1)

# X_train = X_train / np.max(X_train)
# X_valid = X_valid / np.max(X_valid)
xtrain = np.array(X_train)
xvalid = np.array(X_valid)

In [ ]:
xtrain, xvalid

In [ ]:

# def objective(trial):
#     n_unit0 = trial.suggest_int('units0', 50, 500)
#     n_unit1 = trial.suggest_int('units1', 50, 500)
#     n_unit2 = trial.suggest_int('units2', 50, 500)
#     dropout = trial.suggest_float('dropout', 0.0, 0.9)
#     lr = trial.suggest_float('lr', 1e-4, 0.01)
# #     beta_1 = trial.suggest_float('beta_1', 0.0, 1.0)
# #     beta_2 = trial.suggest_float('beta_2', 0.0, 1.0)
# #     epsilon = trial.suggest_float('epsilon', 1e-9, 1e-5)
# #     dropout = trial.suggest_float('dropout', 0.1, 0.9999)
#     model = models.Sequential()
#     model.add(layers.Dense(n_unit0, activation='tanh', input_shape=(X_train.shape[1],)))
#     model.add(layers.Dense(n_unit1, activation='tanh'))
#     model.add(layers.Dense(n_unit2, activation='tanh'))
#     # model.add(layers.Dense(400, activation='relu'))
#     model.add(layers.Dropout(dropout))
#     model.add(layers.Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy',
#                  optimizer='adam',
#                  metrics=['accuracy'])
#     # model.summary()
#     history = model.fit(xtrain, y_train, validation_data=(xvalid, y_valid), epochs=5, batch_size=150,
#                        callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max'))
#     return history.history['val_loss'][len(history.history['val_loss'])-1]
# study = optuna.create_study()
# study.optimize(objective, n_trials=30)

In [ ]:
# study.best_params, study.best_value

In [ ]:
X_train.shape

In [ ]:
model = models.Sequential()
model.add(layers.Dense(2*X_train.shape[1], activation='tanh', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(2*X_train.shape[1], activation='tanh'))
model.add(layers.Dense(2*X_train.shape[1], activation='tanh'))
model.add(layers.Dense(2*X_train.shape[1], activation='tanh'))
# model.add(layers.Dense(100, activation='tanh'))
# model.add(layers.Dense(100, activation='tanh'))
# model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
# model.summary()
history = model.fit(xtrain, y_train, validation_data=(xvalid, y_valid), epochs=100, batch_size=100)

In [ ]:
# history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, batch_size=50)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc)+1)
plt.plot(epochs, acc, 'r', label='train')
plt.plot(epochs, val_acc, 'b', label='validation')
plt.legend()

In [ ]:
model.evaluate(X_valid,y_valid)

In [ ]:
trainCopy.columns

In [ ]:
# testCopy = testCopy / np.max(testCopy)
# trainCopy = np.expand_dims(-1)

# test_cont = testCopy[cont_cols] / np.max(testCopy[cont_cols])
# prediction = model.predict([test_cat, test_cont])
prediction = model.predict(testCopy)
prediction

In [ ]:
p = [r[0] for r in prediction]
p

In [ ]:
output = pd.DataFrame({'id': test_df.id, 'target': p})
output.to_csv('submission.csv', index=False)